In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
plt.rcParams['font.family']='AppleGothic'
plt.rcParams['axes.unicode_minus']=False

# 데이터 수집 및 전처리
## 데이터 수집
- 국토교통부 실거래가 공개시스템<br>
    http://rtdown.molit.go.kr/

In [3]:
df = pd.read_csv('../data/아파트(매매)__실거래가_20220608063909.csv')
df

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,거래유형,중개사소재지
0,강원도 강릉시 견소동,202,202,0,송정한신,59.800,202203,7,"18,000",3,1997.0,경강로2539번길 8,NaN,중개거래,강원 강릉시
1,강원도 강릉시 견소동,202,202,0,송정한신,59.800,202203,8,"21,000",8,1997.0,경강로2539번길 8,NaN,중개거래,강원 강릉시
2,강원도 강릉시 견소동,202,202,0,송정한신,84.945,202203,26,"25,500",10,1997.0,경강로2539번길 8,NaN,중개거래,강원 강릉시
3,강원도 강릉시 견소동,289,289,0,송정해변신도브래뉴아파트,84.990,202203,14,"33,500",1,2005.0,경강로2539번길 22,NaN,중개거래,강원 강릉시
4,강원도 강릉시 견소동,289,289,0,송정해변신도브래뉴아파트,59.990,202203,18,"27,000",13,2005.0,경강로2539번길 22,NaN,중개거래,강원 강릉시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31388,충청북도 충주시 호암동,1191,1191,0,호암힐데스하임,99.870,202203,12,"55,500",4,2019.0,호암토성2로 1,20220419.0,중개거래,충북 충주시
31389,충청북도 충주시 호암동,1191,1191,0,호암힐데스하임,120.490,202203,18,"69,000",19,2019.0,호암토성2로 1,NaN,직거래,-
31390,충청북도 충주시 호암동,1191,1191,0,호암힐데스하임,84.950,202203,19,"47,800",20,2019.0,호암토성2로 1,NaN,중개거래,충북 충주시
31391,충청북도 충주시 호암동,1191,1191,0,호암힐데스하임,84.890,202203,26,"48,800",24,2019.0,호암토성2로 1,NaN,중개거래,충북 충주시


## 데이터 전처리
### 해제사유발생일
- 해제사유 발생일:계약이 해지된 것에 대한 해지일

In [4]:
# 해제사유 발생일이 존재하는 데이터 삭제
df = df.drop(index=df[df['해제사유발생일'].notnull()].index)

In [5]:
df[df['해제사유발생일'].notnull()]

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,거래유형,중개사소재지


### 사용할 컬럼만 추출
- 시군구, 전용면적(㎡), 거래금액(만원)

In [6]:
df = df[['시군구', '전용면적(㎡)', '거래금액(만원)']]
df.head()

,시군구,전용면적(㎡),거래금액(만원)
0,강원도 강릉시 견소동,59.800,"18,000"
1,강원도 강릉시 견소동,59.800,"21,000"
2,강원도 강릉시 견소동,84.945,"25,500"
3,강원도 강릉시 견소동,84.990,"33,500"
4,강원도 강릉시 견소동,59.990,"27,000"


### 자료형 확인 및 변경

In [7]:
df.dtypes

시군구          object
전용면적(㎡)     float64
거래금액(만원)     object
dtype: object

In [8]:
# 전용면적: int(소수점 무시)
df['전용면적(㎡)'] = df['전용면적(㎡)'].astype('int64')

In [9]:
# 거래금액 : int
df['거래금액(만원)'] = df['거래금액(만원)'].str.replace(',','').astype('int64')

In [10]:
df.dtypes

시군구         object
전용면적(㎡)      int64
거래금액(만원)     int64
dtype: object